In [19]:
import pandas as pd
import psycopg2
from psycopg2 import sql


In [22]:
def create_table_and_ingest_data(db_params, table_name, excel_file_path):
    connection = None
    cursor = None

    try:
        # Read the Excel file into a DataFrame
        df = pd.read_excel(excel_file_path)
        print("Excel file read successfully.")
        print(df.head())  # Display the first few rows of the DataFrame for verification

        # Convert boolean columns to integers
        bool_cols = df.select_dtypes(include='bool').columns
        df[bool_cols] = df[bool_cols].astype(int)

        # Establish a connection to the database
        connection = psycopg2.connect(**db_params)
        cursor = connection.cursor()

        # Drop the table if it exists
        drop_table_query = sql.SQL("DROP TABLE IF EXISTS {table}").format(
            table=sql.Identifier(table_name)
        )
        cursor.execute(drop_table_query)
        connection.commit()

        # Dynamically create table based on DataFrame columns
        create_table_query = sql.SQL(
            "CREATE TABLE {table} ({fields})"
        ).format(
            table=sql.Identifier(table_name),
            fields=sql.SQL(', ').join(
                sql.SQL("{} {}").format(
                    sql.Identifier(col),
                    sql.SQL("TEXT") if df[col].dtype == 'object' else
                    sql.SQL("INTEGER") if df[col].dtype in ['int64', 'bool'] else
                    sql.SQL("FLOAT") if df[col].dtype == 'float64' else
                    sql.SQL("TIMESTAMP") if df[col].dtype == 'datetime64[ns]' else
                    sql.SQL("TEXT")
                ) for col in df.columns
            )
        )
        cursor.execute(create_table_query)
        connection.commit()

        # Build the insert query dynamically based on the DataFrame columns
        insert_query = sql.SQL('INSERT INTO {table} ({fields}) VALUES ({values})').format(
            table=sql.Identifier(table_name),
            fields=sql.SQL(', ').join(map(sql.Identifier, df.columns)),
            values=sql.SQL(', ').join(sql.Placeholder() * len(df.columns))
        )
        
        # Execute insert queries
        for row in df.itertuples(index=False, name=None):
            cursor.execute(insert_query, row)
        
        # Commit the transaction
        connection.commit()

        print("Data ingested successfully.")
    except FileNotFoundError as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        # Close the cursor and connection if they were created
        if cursor is not None:
            cursor.close()
        if connection is not None:
            connection.close()

In [23]:
db_params = {
    'dbname': 'data',        # Replace with your actual database name
    'user': 'postgres',        # Replace with your actual username
    'password': '1611',    # Replace with your actual password
    'host': 'localhost',            # Adjust if your database server is not local
    'port': '5432'                  # Default PostgreSQL port
}

# Define the table name and the path to the Excel file
table_name = 'details'
excel_file_path = r'C:\Users\sastr\OneDrive\Desktop\Book.xlsx'

# Call the function to create the table and ingest data
create_table_and_ingest_data(db_params, table_name, excel_file_path)

Excel file read successfully.
   id  age  height  gender initial              joined
0   1   25     5.9    True       A 2023-01-15 10:23:00
1   2   30     6.1   False       B 2023-02-20 14:45:00
2   3   22     5.5    True       C 2023-03-10 09:12:00
3   4   35     6.0    True       D 2023-04-25 16:30:00
4   5   28     5.8   False       E 2023-05-05 11:15:00
Data ingested successfully.
